## Data Procesing

In [1]:
import pandas as pd

In [2]:
import h2o
from h2o.estimators.random_forest import H2ORandomForestEstimator
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,3 hours 12 mins
H2O_cluster_timezone:,Asia/Colombo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.2
H2O_cluster_version_age:,2 months and 16 days
H2O_cluster_name:,H2O_from_python_Asus_e6utam
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.972 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [3]:
%%time
train_new = pd.read_csv("D:\\DC Universe\\Ucsc\\Third Year\\ENH 3201 Industrial Placements\\H20 Applications\\H20 ML Notebooks\\H20Csv\\Titanic\\titanic_train.csv")
test = pd.read_csv("D:\\DC Universe\\Ucsc\\Third Year\\ENH 3201 Industrial Placements\\H20 Applications\\H20 ML Notebooks\\H20Csv\\Titanic\\titanic_test.csv")
subs = pd.read_csv('D:\\DC Universe\\Ucsc\\Third Year\\ENH 3201 Industrial Placements\\H20 Applications\\H20 ML Notebooks\\H20Csv\\Titanic\\gender_submission.csv')

#train_new.to_pickle("D:\\DC Universe\\Ucsc\\Third Year\\ENH 3201 Industrial Placements\\H20 Applications\\H20 ML Notebooks\\H20Csv\\Titanic\\train_set.pkl")

Wall time: 31.5 ms


In [4]:
%%time
train = pd.read_pickle("D:\\DC Universe\\Ucsc\\Third Year\\ENH 3201 Industrial Placements\\H20 Applications\\H20 ML Notebooks\\H20Csv\\Titanic\\train_set.pkl")

Wall time: 4.06 ms


In [5]:
drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp','Parch']
train = train.drop(drop_elements, axis = 1)
test = test.drop(drop_elements, axis = 1)

def checkNull_fillData(df):
    for col in df.columns:
        if len(df.loc[df[col].isnull() == True]) != 0:
            if df[col].dtype == "float64" or df[col].dtype == "int64":
                df.loc[df[col].isnull() == True,col] = df[col].mean()
            else:
                df.loc[df[col].isnull() == True,col] = df[col].mode()[0]
                
checkNull_fillData(train)
checkNull_fillData(test)

str_list = [] 
num_list = []
for colname, colvalue in train.iteritems():
    if type(colvalue[1]) == str:
        str_list.append(colname)
    else:
        num_list.append(colname)
        
train = pd.get_dummies(train, columns=str_list)
test = pd.get_dummies(test, columns=str_list)

train = h2o.H2OFrame(train)
test = h2o.H2OFrame(test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [6]:
train.describe()

Rows:891
Cols:9




,Survived,Pclass,Age,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
type,int,int,real,real,int,int,int,int,int
mins,0.0,1.0,0.42,0.0,0.0,0.0,0.0,0.0,0.0
mean,0.3838383838383838,2.3086419753086447,29.699117647058795,32.20420796857465,0.35241301907968575,0.6475869809203143,0.18855218855218855,0.08641975308641975,0.7250280583613917
maxs,1.0,3.0,80.0,512.3292,1.0,1.0,1.0,1.0,1.0
sigma,0.4865924542648575,0.8360712409770491,13.002015226002891,49.69342859718089,0.4779900708960982,0.4779900708960982,0.3913721645054733,0.28114069214170423,0.44675091003414663
zeros,549,0,0,15,577,314,723,814,245
missing,0,0,0,0,0,0,0,0,0
0,0.0,3.0,22.0,7.25,0.0,1.0,0.0,0.0,1.0
1,1.0,1.0,38.0,71.2833,1.0,0.0,1.0,0.0,0.0
2,1.0,3.0,26.0,7.925,1.0,0.0,0.0,0.0,1.0


In [7]:
test.describe()

Rows:418
Cols:8




,Pclass,Age,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
type,int,real,real,int,int,int,int,int
mins,1.0,0.17,0.0,0.0,0.0,0.0,0.0,0.0
mean,2.2655502392344493,30.27259036144579,35.627188489208606,0.36363636363636365,0.6363636363636364,0.24401913875598086,0.11004784688995216,0.645933014354067
maxs,3.0,76.0,512.3292,1.0,1.0,1.0,1.0,1.0
sigma,0.8418375519640504,12.634534168325061,55.840500479541056,0.4816221409322308,0.4816221409322308,0.4300188157360399,0.3133244005170708,0.4788026786626084
zeros,0,0,2,266,152,316,372,148
missing,0,0,0,0,0,0,0,0
0,3.0,34.5,7.8292,0.0,1.0,0.0,1.0,0.0
1,3.0,47.0,7.0,1.0,0.0,0.0,0.0,1.0
2,2.0,62.0,9.6875,0.0,1.0,0.0,1.0,0.0


In [8]:
train1, valid1, new_data1 = train.split_frame(ratios = [.7, .15], seed = 1234)

In [9]:
#train.columns
test.columns

['Pclass',
 'Age',
 'Fare',
 'Sex_female',
 'Sex_male',
 'Embarked_C',
 'Embarked_Q',
 'Embarked_S']

In [10]:
predictors = ["Age","Embarked_C","Pclass","Embarked_Q","Sex_male"]
response = "Fare"

In [11]:
titanic = H2ORandomForestEstimator(model_id="titanic", ntrees = 1, seed = 1234)
titanic.train(x = predictors, y = response, training_frame = train1, validation_frame = valid1)

drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  titanic


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,1.0,1.0,1801.0,15.0,15.0,15.0,139.0,139.0,139.0




ModelMetricsRegression: drf
** Reported on train data. **

MSE: 1133.3550884778363
RMSE: 33.6653395716995
MAE: 17.77549033043748
RMSLE: 0.7013625743352557
Mean Residual Deviance: 1133.3550884778363

ModelMetricsRegression: drf
** Reported on validation data. **

MSE: 908.6602305039935
RMSE: 30.14399161531189
MAE: 18.11057029921433
RMSLE: 0.7471205311672524
Mean Residual Deviance: 908.6602305039935

Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
0,,2022-04-11 14:41:32,0.011 sec,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,,2022-04-11 14:41:32,0.191 sec,1.0,33.66534,17.77549,1133.355088,30.143992,18.11057,908.660231



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,Pclass,497683.031250,1.000000,0.855305
1,Embarked_C,61223.972656,0.123018,0.105218
2,Age,19492.304688,0.039166,0.033499
3,Sex_male,3118.316406,0.006266,0.005359
4,Embarked_Q,360.259369,0.000724,0.000619


In [12]:
model_path = h2o.save_model(model=titanic, path="D:\\DC Universe\\Ucsc\\Third Year\\ENH 3201 Industrial Placements\\H20 Applications\\H20 ML Notebooks\\H20Csv\\Titanic\\", force=True)
print (model_path)

D:\DC Universe\Ucsc\Third Year\ENH 3201 Industrial Placements\H20 Applications\H20 ML Notebooks\H20Csv\Titanic\titanic


In [13]:
titanic = h2o.load_model("D:\\DC Universe\\Ucsc\\Third Year\\ENH 3201 Industrial Placements\\H20 Applications\\H20 ML Notebooks\\H20Csv\\Titanic\\titanic")

In [14]:
titanic

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  titanic


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,1.0,1.0,1801.0,15.0,15.0,15.0,139.0,139.0,139.0




ModelMetricsRegression: drf
** Reported on train data. **

MSE: 1133.3550884778363
RMSE: 33.6653395716995
MAE: 17.77549033043748
RMSLE: 0.7013625743352557
Mean Residual Deviance: 1133.3550884778363

ModelMetricsRegression: drf
** Reported on validation data. **

MSE: 908.6602305039935
RMSE: 30.14399161531189
MAE: 18.11057029921433
RMSLE: 0.7471205311672524
Mean Residual Deviance: 908.6602305039935

Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
0,,2022-04-11 14:41:32,0.011 sec,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,,2022-04-11 14:41:32,0.191 sec,1.0,33.66534,17.77549,1133.355088,30.143992,18.11057,908.660231



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,Pclass,497683.031250,1.000000,0.855305
1,Embarked_C,61223.972656,0.123018,0.105218
2,Age,19492.304688,0.039166,0.033499
3,Sex_male,3118.316406,0.006266,0.005359
4,Embarked_Q,360.259369,0.000724,0.000619


In [15]:
train2, valid2, new_data2 = test.split_frame(ratios = [.7, .15], seed = 1234)

In [16]:
# Checkpoint on the same dataset. This shows how to train an additional
# 9 trees on top of the first 1. To do this, set ntrees equal to 10.
titanic_continued = H2ORandomForestEstimator(model_id = 'titanic_new',
                                         checkpoint = titanic,
                                         ntrees = 4,
                                         seed = 1234)
titanic_continued.train(x = predictors, y = response, training_frame = train2, validation_frame = valid2)

drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  titanic_new


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,4.0,4.0,5764.0,13.0,20.0,12.25,85.0,139.0,75.5




ModelMetricsRegression: drf
** Reported on train data. **

MSE: 2146.237534249252
RMSE: 46.327503000369575
MAE: 20.323413028791258
RMSLE: 0.6753339178022078
Mean Residual Deviance: 2146.237534249252

ModelMetricsRegression: drf
** Reported on validation data. **

MSE: 1937.71312079991
RMSE: 44.01946297718669
MAE: 25.51143854038388
RMSLE: 0.9055540088172322
Mean Residual Deviance: 1937.71312079991

Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
0,,2022-04-11 14:41:32,-28.-575 sec,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,,2022-04-11 14:41:32,-28.-395 sec,1.0,50.878531,21.325502,2588.624951,44.124799,25.311824,1946.997898
2,,2022-04-11 14:42:01,0.259 sec,2.0,49.565239,21.452445,2456.712907,46.402843,26.467226,2153.223877
3,,2022-04-11 14:42:01,0.291 sec,3.0,47.453296,20.112071,2251.815287,45.073059,26.415759,2031.580672
4,,2022-04-11 14:42:01,0.501 sec,4.0,46.327503,20.323413,2146.237534,44.019463,25.511439,1937.713121



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,Pclass,641299.187500,1.000000,0.404932
1,Age,632948.687500,0.986979,0.399660
2,Embarked_C,200252.921875,0.312261,0.126445
3,Sex_male,77170.390625,0.120334,0.048727
4,Embarked_Q,32047.763672,0.049973,0.020236


In [17]:
col_count=len(train.columns)
col_count

9

In [18]:
row_count=len(train)
# split_ratio = int(input("Enter Split Ratio >>>"))

In [19]:
#split_ratio = int(input("Enter Split Ratio >>>"))
#import modin.pandas as mpd